### Загружаем базовую модель

In [1]:
from vlm_finetune import AutoVlmModel, ImageProcessor


model = AutoVlmModel.from_name(
    model_name="qwen", 
    model_path="Qwen/Qwen3-VL-8B-Thinking",
    image_processor=ImageProcessor(image_size=(336, 336)) # можно заресайзить картинки, чтобы сэкономить память
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

### Тестим на нашем датасете, для примера возьмём рандомный сэмпл

In [2]:
import json

with open("data/vet_dataset_test_eng.json", "r") as file:
    sample = json.loads(file.read())[0]

In [3]:
sample

{'image_path': 'data/грудная/52а.png',
 'answer': 'the lungs are straightened without focal and infiltrated shadows. the pulmonary pattern is not changed. the shadow of the heart and aorta is unchanged. the tracheal lumen is not broken. compaction of the cartilage of the larynx, increase in volume.  At the time of the study, signs of laryngitis were revealed'}

In [4]:
prompt = "Make a probable diagnosis using the x-ray."

In [5]:
%%time

model_answer = model.predict(image=sample["image_path"], prompt=prompt)

CPU times: user 19.8 s, sys: 5.7 s, total: 25.5 s
Wall time: 26 s


In [6]:
model_answer

"<think>\nSo, let's look at the X-ray image. It's a radiograph of a dog, right? The skeleton is visible, so we need to check for abnormalities. First, let's identify the parts: the skull, spine, ribs, limbs. Wait, the X-ray shows the dog lying down, so the spine is straight, but maybe the limbs? Wait, no—maybe the spine has a curvature? Wait, no, the question is about a probable diagnosis. Let's think about common skeletal issues in dogs.\n\nWait, looking at the image, the spine might have a deviation. Wait, in X-rays, if there's a spinal deformity, like scoliosis, which is a sideways curvature of the spine. But let's check: the dog's spine, in the lateral view (since it's a side view), the spine should be straight. If it's curved, that's scoliosis. But also, maybe a fracture? Wait, no, the bones look intact. Wait, another possibility: degenerative myelopathy? No, that's neurological, not seen on X-ray. Wait, osteoarthritis? But that's joint issues, which would show narrowing of joints

In [7]:
import evaluate

bertscore = evaluate.load("bertscore")

In [8]:
bertscore.compute(predictions=[model_answer], references=[sample["answer"]], lang="ru")

{'precision': [0.5914313793182373],
 'recall': [0.6676635146141052],
 'f1': [0.6272397041320801],
 'hashcode': 'bert-base-multilingual-cased_L9_no-idf_version=0.3.12(hug_trans=4.57.1)'}

**Модель ответила не совсем по теме, много длинных размышлений**

### Зафайнтюним базовую модель с помощью фреймворка
Параметры берём дефолтные (они уже зашиты во фреймворк), параметры можно менять и "докручивать"

In [9]:
model.finetune(
    dataset_path="data/vet_dataset_train_eng.json", 
    output_dir="models/qwen_tuned_eng", 
    prompt=prompt, 
    num_train_epochs=3
)

2025-11-02 11:40:15 [INFO] vlm_finetune.base.model: Используется устройство: cuda:0
2025-11-02 11:40:15 [INFO] vlm_finetune.base.model: Количество обучаемых параметров: 15335424
2025-11-02 11:40:15 [INFO] vlm_finetune.base.model: Начинаем finetune...


trainable params: 15,335,424 || all params: 8,782,459,120 || trainable%: 0.1746


Step,Training Loss
10,3.303600
20,2.102100


2025-11-02 12:32:49 [INFO] vlm_finetune.base.model: Модель успешно дообучена и сохранена в models/qwen_tuned_eng
2025-11-02 12:32:49 [INFO] vlm_finetune.base.model: !Для предсказаний будет использоваться дообученная модель!


### Смотрим тот же сэмпл из нашего датасета

In [10]:
%%time


model_answer = model.predict(image=sample["image_path"], prompt=prompt)

CPU times: user 19.9 s, sys: 5.35 s, total: 25.2 s
Wall time: 25 s


In [11]:
model_answer

'<think>\n</think>\n\nX-ray findings: 1. Normal heart size and shape. 2. No signs of aseptic and purulent processes, pneumosclerosis. 3. Lungs - a homogeneous shadow, without focal shadows, signs of the infiltrative form of the disease. 4. Bones of the ribs, limbs, spine - no signs of pathology. 5. No signs of pneumothorax. 6. No signs of heart failure. 7. Tracheobronchial tree - no signs of foreign bodies, narrowing of the tracheobronchial tree. 8. The larynx is not changed. Conclusion: the radiological picture is consistent with the normal state of the chest.'

In [12]:
bertscore.compute(predictions=[model_answer], references=[sample["answer"]], lang="ru")

{'precision': [0.6758347153663635],
 'recall': [0.7459601163864136],
 'f1': [0.709168016910553],
 'hashcode': 'bert-base-multilingual-cased_L9_no-idf_version=0.3.12(hug_trans=4.57.1)'}

**Модель отвечает по теме**